## Influence de l'environnement culturel sur le vote RN

# Contexte

A DEVELOPPER 

Etape préliminaire

Avant de commencer le projet, il faut installer les packages dont nous allons avoir besoin

In [1]:
import requests
import pandas as pd
import zipfile
import os

Etape 1 : Importation des données

1) Importation de la base données électorales

In [2]:
# DONNÉES INFRASTRUCTURES (S3 SSPCloud)

# Chemin S3 du fichier
path_infra_s3 = "s3://hantier/DS_BPE_2024_data.csv"

# Lecture directe depuis S3
df_infrastruct = pd.read_csv("s3://hantier/DS_BPE_2024_data.csv", sep=';')  # le séparateur utilisé est un ;
print("Le dataframe df_infrastruct peut être traité comme un data frame classique avec pandas")

Le dataframe df_infrastruct peut être traité comme un data frame classique avec pandas


/tmp/ipykernel_8962/4256203287.py:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_infrastruct = pd.read_csv("s3://hantier/DS_BPE_2024_data.csv", sep=';')  # le séparateur utilisé est un ;


2) Importation de la base infrastructures


Afin que cette étape se déroule bien, il faut récupérer la base DS_BPE_2024_data.csv préalablement importée sur S3.

In [ ]:
url_elect = "https://www.data.gouv.fr/api/1/datasets/r/6d9b33e5-667d-4c3e-9a0b-5fdf5baac708"
path_elect = "/home/onyxia/work/Projet-Python-S3/code/results_elect.xlsx"

# Téléchargement du fichier Excel
response = requests.get(url_elect)
response.raise_for_status()

with open(path_elect, "wb") as f:
    f.write(response.content)

# Lecture du fichier Excel
df_extrait = pd.read_excel(path_elect, sheet_name=0)

print("Le dataframe df_extrait peut être traité comme un data frame classique avec pandas")

print("Informations sur le dataframe df_extrait qui est notre df initial")
print(df_extrait.columns)
print(len(df_extrait))


# la colonne unamed 53 correspond au nombre de voix de JLF, la colonne unamed 67 correspond à celui de marine le pen
liste_var = ['Code du département', 
    'Libellé du département', 
    'Code de la commune', 
    'Libellé de la commune',
    'Inscrits', 
    'Abstentions',
    '% Abs/Ins',
    'Unnamed: 53', 
    'Unnamed: 67']
df_extrait2 = df_extrait[liste_var]
df_extrait3 = df_extrait2.rename(columns={'Unnamed: 53': 'score_rn'})
df_elect = df_extrait3.rename(columns={'Unnamed: 67': 'score_lfi'})
print("Informations sur le dataframe df_elect")
print(df_elect.columns)
print(len(df_elect))
print(df_elect.head(4))

Le dataframe df_extrait peut être traité comme un data frame classique avec pandas
Informations sur le dataframe df_extrait qui est notre df initial
Index(['Code du département', 'Libellé du département', 'Code de la commune',
       'Libellé de la commune', 'Etat saisie', 'Inscrits', 'Abstentions',
       '% Abs/Ins', 'Votants', '% Vot/Ins',
       ...
       'Unnamed: 93', 'Unnamed: 94', 'Unnamed: 95', 'Unnamed: 96',
       'Unnamed: 97', 'Unnamed: 98', 'Unnamed: 99', 'Unnamed: 100',
       'Unnamed: 101', 'Unnamed: 102'],
      dtype='object', length=103)
35245
Informations sur le dataframe df_elect
Index(['Code du département', 'Libellé du département', 'Code de la commune',
       'Libellé de la commune', 'Inscrits', 'Abstentions', '% Abs/Ins',
       'score_rn', 'score_lfi'],
      dtype='object')
35245
  Code du département Libellé du département  Code de la commune  \
0                  01                    Ain                   1   
1                  01                    Ai

2.1) Statistiques descriptives

Statistiques descriptives sur df_elect

In [17]:
print("Le score moyen par commune du RN était")
print(df_elect["score_rn"].mean())
print(df_elect["score_rn"].median())
print("Le score moyen par commune de LFI était")
print(df_elect["score_lfi"].mean())
print(df_elect["score_lfi"].median())

Le score moyen par commune du RN était
29.075128387005247
28.85
Le score moyen par commune de LFI était
16.511555681656972
15.57


2) Préparation des données infrastructures
Dans un second temps, nous devons préparer le dataframe des infrastructures. Nous allons le passer d'un format long à un format large, autrement dit, au lieu d'avoir une ligne par infrastructure, nous aurons une ligne par ville et une colonne par infrastructure.

In [5]:
print("Informations sur le dataframe df_infrastruct")
print(df_infrastruct.columns)
print(df_infrastruct.size)
print(df_infrastruct.head())

Informations sur le dataframe df_infrastruct
Index(['GEO', 'GEO_OBJECT', 'FACILITY_DOM', 'FACILITY_SDOM', 'FACILITY_TYPE',
       'BPE_MEASURE', 'TIME_PERIOD', 'OBS_VALUE'],
      dtype='object')
18403840
     GEO GEO_OBJECT FACILITY_DOM FACILITY_SDOM FACILITY_TYPE BPE_MEASURE  \
0  29305     UU2020            D            D1          D115  FACILITIES   
1  77205     UU2020            D            D2          D265  FACILITIES   
2  31113     UU2020            D            D2          D249  FACILITIES   
3  41501     UU2020            D            D1          D112  FACILITIES   
4  46104     UU2020            D            D2          D266  FACILITIES   

   TIME_PERIOD  OBS_VALUE  
0         2024          1  
1         2024          4  
2         2024          1  
3         2024          1  
4         2024          1  


In [6]:
df_infra_large = (
    df_infrastruct
    .query("TIME_PERIOD == 2024")
    .groupby(["GEO", "FACILITY_TYPE"], as_index=False)["OBS_VALUE"]
    .sum()
    .pivot(index="GEO", columns="FACILITY_TYPE", values="OBS_VALUE")
    .fillna(0)
    .reset_index()
)


In [7]:
print("Vérifications sur le dataframe df_infra_large")
print(df_infra_large.columns)
print(df_infra_large.size)
print(df_infra_large.head())

Vérifications sur le dataframe df_infra_large
Index(['GEO', 'A101', 'A104', 'A105', 'A108', 'A109', 'A120', 'A121', 'A122',
       'A124',
       ...
       'F312', 'F313', 'F314', 'F315', 'G101', 'G102', 'G103', 'G104', 'G105',
       '_T'],
      dtype='object', name='FACILITY_TYPE', length=231)
15345330
FACILITY_TYPE GEO  A101  A104  A105  A108  A109  A120  A121  A122  A124  ...  \
0               1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
1               2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
2               3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
3               4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
4               6   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   

FACILITY_TYPE  F312  F313  F314  F315  G101  G102  G103  G104  G105       _T  
0               0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  56142.0  
1               0.0   0.0   0.0   0.0   0.0   0.0   0